In [1]:
import networkx as nx
import pandas as pd
import json

from exceptiongroup import catch
from networkx.exception import NetworkXError

In [2]:
# graphs = ["stexpanded", "memoryalpha", "swtor", "swg", "memorybeta", "marvel", "mcu", "starwars"]
graphs = ["stexpanded"]
community_algorithm = "bigclam"

### Egy nagy és pár kicsi komponens.

In [ ]:
for graph in graphs:

    print(graph)

    graph_file = "data/filtered_triples_weighted/" + graph + ".triples"

    graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
    G = nx.Graph()
    for row in graph_df.itertuples():
        G.add_edge(int(row[1]), int(row[2]))
        # G.add_edge(int(row[1]), int(row[2]), weight=int(row[3]))

    components = nx.connected_components(G)
    for comp in components:
        print(comp)
        print(len(list(comp)))

    print(len(G.nodes))
    G.remove_nodes_from([58281, 6266, 72485, 24685])
    print(len(G.nodes))

    print(nx.diameter(G))

### Louvain közösségkereső: nagy gráfra nagyon kevés közösséget ad

In [ ]:
graph = "memoryalpha"

graph_file = "data/filtered_triples_weighted/" + graph + ".triples"
communities_file = "./results/communities_" + community_algorithm + "/" + graph + "_10000.txt"

graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
G = nx.Graph()
for row in graph_df.itertuples():
    G.add_edge(int(row[1]), int(row[2]))
    # G.add_edge(int(row[1]), int(row[2]), weight=int(row[3]))

communities = nx.community.louvain_communities(G)
print(communities)

### Közösségkeresés, bennük fokszámeloszlás

In [9]:
for graph in graphs:

    print(graph)
    print("##################################################")

    pair = "memoryalpha"

    graph_file = "data/filtered_triples_weighted/" + graph + ".triples"
    communities_file = "./results/communities_" + community_algorithm + "/" + graph + "_10000.txt"
    mappings_file = "./data/triples_v2/" + graph + "_mapping.json"
    exact_match_file = "./data/exact_match/" + pair + "-" + graph + "-formatted.json"

    graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
    G = nx.Graph()
    for row in graph_df.itertuples():
        G.add_edge(int(row[1]), int(row[2]))
        # G.add_edge(int(row[1]), int(row[2]), weight=int(row[3]))

    communities = []
    with open(communities_file) as cf:
        for line in cf:
            numbers_set = {int(num) for num in line.strip().split("\t")}
            communities.append(numbers_set)

    with open(mappings_file) as mf:
        mappings = json.load(mf)

    mappings_dict = {value: key for key, value in mappings.items()}

    with open(exact_match_file) as file:
        exact_match = json.load(file)

    for community in communities:
        print("community size:", len(community))
        G_sub = G.subgraph(community)
        degrees = G_sub.degree()
        degree_sequence = sorted((d for n, d in degrees), reverse=True)
        node_with_highest_degree = max(degrees, key=lambda x: x[1])
        print("highest degree:", node_with_highest_degree[1])
        print(mappings_dict[node_with_highest_degree[0]])
        # print(mappings_dict[node_with_highest_degree[0]], "---", node_with_highest_degree[1])
        # print(degree_sequence)
        # print(node_with_highest_degree)
        try:
            diameter = nx.diameter(G_sub)
            print("diameter:", nx.diameter(G_sub))
        except nx.NetworkXError:
            pass
        count_exact_match = 0
        for node in community:
            for em in exact_match:
                if em[1] == node:
                    count_exact_match += 1
                    # print("match found:", node, "---", em[0])
        print("exact match count:", count_exact_match)
        print("##################################################")

stexpanded
##################################################
community size: 1655
highest degree: 1655
http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Starfleet
diameter: 3
exact match count: 616
##################################################
community size: 785
highest degree: 782
http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Category:Humans
diameter: 3
exact match count: 103
##################################################
community size: 573
highest degree: 530
http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/RIS_Bouteina_(fan_fiction_series)
diameter: 5
exact match count: 258
##################################################
community size: 473
highest degree: 425
http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Star_Trek:_Pendragon
diameter: 3
exact match count: 266
##################################################
community size: 425
highest degree: 386
http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resou